Challenger data analysis

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

Import dataset

In [29]:
orings = pd.read_csv('o-ring-erosion-only.data', sep=' ', 
                     header=None, index_col=-1,
                     names=['total', 'eroded', 'temperature', 'pressure', ''])
orings.head()

,total,eroded,temperature,pressure
,,,,
1,6,0,66,50
2,6,1,70,50
3,6,0,69,50
4,6,0,68,50
5,6,0,67,50


Code for forming design matrix.